In [18]:
import ee
import geemap.foliumap as geemap 
import ssl
import webbrowser

ssl._create_default_https_context = ssl._create_stdlib_context
ee.Authenticate()
ee.Initialize(project='ee-juliaszymanska142')

# Define Area of Interest (AOI)
aoi = ee.Geometry.Rectangle([19.6, 52.4, 20.0, 52.8])


Map2 = geemap.Map()
Map2.centerObject(aoi, zoom=10)

In [19]:
image1 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate('2023-04-01', '2023-04-03') \
    .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
    .first()

image2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate('2023-07-01', '2023-07-03') \
    .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
    .first()


vis_params = {
    'bands': ['B4', 'B3', 'B2'],  
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}


Map2.addLayer(image1, vis_params, 'Sentinel-2 (2023-04-01)')
Map2.addLayer(image2, vis_params, 'Sentinel-2 (2023-07-01)')
Map2.addLayer(aoi, {'color': 'red'}, 'AOI')

Map2.save("map2.html")
webbrowser.open("map2.html")

True

In [20]:
ndvi1 = image1.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndvi2 = image2.normalizedDifference(['B8', 'B4']).rename('NDVI')

ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

Map2.addLayer(ndvi1, ndvi_vis, 'NDVI (2023-04-01)')
Map2.addLayer(ndvi2, ndvi_vis, 'NDVI (2023-07-01)')

Map2.save("NDVI.html")
webbrowser.open("NDVI.html")

True

In [21]:
if image1 and image2:
    ndvi_diff = ndvi2.subtract(ndvi1).rename('NDVI Change')

    ndvi_diff_vis = {
        'min': -0.5,
        'max': 0.5,
        'palette': ['red', 'white', 'green']
    }

    Map2.addLayer(ndvi_diff, ndvi_diff_vis, 'NDVI Change (2023-04 to 2023-07)')

  
    stats = ndvi_diff.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=30,
        bestEffort=True
    )

    print("The average difference of NDVI:", stats.getInfo())

Map2.save("Difference.html")
webbrowser.open("Difference.html")

The average difference of NDVI: {'NDVI Change': 0.28811659101413234}


True

The NDVI analysis showed positive changes in many areas, indicating an improvement in vegetation conditions over the analyzed period. This is likely due to the natural growth of plants during the summer season. The eastern part of the map appears completely white, signifying a lack of data, which prevents a detailed analysis of NDVI changes in this region. The most probable causes of this gap are dense cloud cover during satellite data acquisition or limited availability of Sentinel-2 images for this part of the AOI.

In [22]:
vegetation_viz = {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 4000}

Map2.addLayer(image1, vegetation_viz, 'Vegetation (2023-04-01)')
Map2.addLayer(image2, vegetation_viz, 'Vegetation (2023-07-01)')

Map2.save("Vegetation.html")
webbrowser.open("Vegetation.html")

True